In [68]:
from Crawling_Dataset import Crawling_Nomal_Dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import cv2 
import os
import numpy as np
from torchvision import transforms
import torch.utils.data as data
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import math

In [69]:
test_path = '/opt/ml/data/celeb_30/cut_test'
train_path = '/opt/ml/data/celeb_30/cut_train'
BATCH_SIZE = 10

In [70]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [71]:
transform = transforms.Compose([
        np.float32,
        transforms.ToTensor(),  # range [0, 255] -> [0.0, 1.0]
        fixed_image_standardization
    ])

In [72]:
test_dataset = Crawling_Nomal_Dataset(test_path, transforms=transform)
train_dataset = Crawling_Nomal_Dataset(train_path, transforms=transform)
test_embedding_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=False)
train_embedding_loader = data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2, drop_last=False)

In [73]:
labels = train_dataset.name_labels

In [74]:
NUM2NAME_dict = train_dataset.get_label_dict_NUM2NAME()
NAME2NUM_dict = train_dataset.get_label_dict_NAME2NUM()
print(NUM2NAME_dict)

{0: 'Jangdoyoun', 1: 'BTSsuga', 2: 'Yimsiwan', 3: 'Shinsekyung', 4: 'Leeseungki', 5: 'Taeyeon', 6: 'Sonhuengmin', 7: 'BTSjhin', 8: 'Yooyeonseok', 9: 'Ahnyoungmi', 10: 'Jojeongseok', 11: 'Chunwoohee', 12: 'Hanhyojoo', 13: 'Jokyuhyun', 14: 'Ohyeonseo', 15: 'Parkhaseon', 16: 'Leegookju', 17: 'SongJoongki', 18: 'Parknarae', 19: 'Parkjiseong'}


In [75]:
model = InceptionResnetV1(
    classify=False,
    pretrained='vggface2'
).to(device)

In [76]:
def get_embedding(feature ) :
    feature = feature.to(device)
    embedding = model(feature)
    embedding = embedding.to('cpu').numpy()
    return embedding

In [77]:
def distance(embeddings1, embeddings2, distance_metric=0):
    if distance_metric==0:
        # Euclidian distance
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),1)
    elif distance_metric==1:
        # Distance based on cosine similarity
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        similarity = dot / norm
        dist = np.arccos(similarity) / math.pi
    else:
        raise 'Undefined distance metric %d' % distance_metric

    return dist

In [67]:
model.eval()
with torch.no_grad():
    for feature, b in train_embedding_loader :
        train_embedding = get_embedding(feature)
        print(train_embedding.shape)
        print(c)
        label_embedding_list = []
        for test_feature, label_list in test_embedding_loader :
            test_embedding = get_embedding(test_feature)
            print(test_embedding.shape)
            dist = distance(train_embedding, test_embedding, 0)
            test_label_nums = test_label_nums.tolist()
            for pair in zip(dist, label_list) :
                label_embedding_list.append(pair)
        label_embedding_list = sorted(label_embedding_list)
        print(len(label_embedding_list))
        print(label_embedding_list[:10])
        break

ValueError: too many values to unpack (expected 2)